In [119]:
#Import de bibliotecas a serem utilizdas
import pandas as pd
import numpy as np
from sqlalchemy.engine import URL,create_engine

In [2]:
#Criação de conexão ao banco de dados
server_name = "DESKTOP-9RK4J8O"
db_name = "AdventureWorksDW2019"
connection_str =  'Driver={SQL Server};' + 'Server=' + server_name + ";Database=" + db_name + ";Trusted_Connection=yes;"

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_str})
engine = create_engine(connection_url)
con = engine.connect()

In [3]:
tabelas = ["FactFinance","DimAccount","DimCurrency","DimOrganization","DimDate","DimDepartmentGroup","DimOrganization","DimScenario","DimCurrency"]

In [8]:
for tb in tabelas:
    tb = pd.read_sql("SELECT * FROM dbo."+tb, con)

In [10]:
#Leitura do banco de dados em um dataframe do pandas
FactFinance = pd.read_sql("SELECT * FROM dbo.FactFinance", con)
FactFinance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39409 entries, 0 to 39408
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   FinanceKey          39409 non-null  int64         
 1   DateKey             39409 non-null  int64         
 2   OrganizationKey     39409 non-null  int64         
 3   DepartmentGroupKey  39409 non-null  int64         
 4   ScenarioKey         39409 non-null  int64         
 5   AccountKey          39409 non-null  int64         
 6   Amount              39409 non-null  float64       
 7   Date                39409 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 2.4 MB


In [11]:
#Leitura das demais tabelas
DimAccount = pd.read_sql("SELECT * FROM dbo.DimAccount", con)
DimDate = pd.read_sql("SELECT * FROM dbo.DimDate", con)
DimDepartmentGroup = pd.read_sql("SELECT * FROM dbo.DimDepartmentGroup", con)
DimOrganization = pd.read_sql("SELECT * FROM dbo.DimOrganization", con)
DimScenario = pd.read_sql("SELECT * FROM dbo.DimScenario", con)
DimCurrency = pd.read_sql("SELECT * FROM dbo.DimCurrency", con)

In [15]:
#Verificando o nomes das colunas das tabelas carregadas
tabelas = [FactFinance,DimAccount,DimCurrency, DimOrganization,DimDate,DimDepartmentGroup,DimOrganization,DimScenario,DimCurrency]
for df in tabelas:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39409 entries, 0 to 39408
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   FinanceKey          39409 non-null  int64         
 1   DateKey             39409 non-null  int64         
 2   OrganizationKey     39409 non-null  int64         
 3   DepartmentGroupKey  39409 non-null  int64         
 4   ScenarioKey         39409 non-null  int64         
 5   AccountKey          39409 non-null  int64         
 6   Amount              39409 non-null  float64       
 7   Date                39409 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 2.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   AccountKey                    

In [12]:
DimCurrency.head()

,CurrencyKey,CurrencyAlternateKey,CurrencyName
0,1,AFA,Afghani
1,2,DZD,Algerian Dinar
2,3,ARS,Argentine Peso
3,4,AMD,Armenian Dram
4,5,AWG,Aruban Guilder


In [4]:
conn = pyodbc.connect(connection_str)

In [13]:
DimOrganization.head()

,OrganizationKey,ParentOrganizationKey,PercentageOfOwnership,OrganizationName,CurrencyKey
0,1,NaN,1,AdventureWorks Cycle,100
1,2,1.0,1,North America Operations,100
2,3,14.0,1,Northeast Division,100
3,4,14.0,1,Northwest Division,100
4,5,14.0,1,Central Division,100


In [5]:
query_FactFinance = "SELECT * FROM dbo.FactFinance"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39409 entries, 0 to 39408
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   FinanceKey          39409 non-null  int64         
 1   DateKey             39409 non-null  int64         
 2   OrganizationKey     39409 non-null  int64         
 3   DepartmentGroupKey  39409 non-null  int64         
 4   ScenarioKey         39409 non-null  int64         
 5   AccountKey          39409 non-null  int64         
 6   Amount              39409 non-null  float64       
 7   Date                39409 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 2.4 MB


#### Irei utilizar apenas o nome da moeda em meu relatório, portanto irei levar essa informação da tabela DimCurrency para a DimOrganization a fim de simplificar o modelo e remover registros desnecessários

In [41]:
#Trazer os dados da currency para organization
DimOrganization_trat = DimOrganization.merge(DimCurrency, how='left', on='CurrencyKey')
DimOrganization_trat.head()

,OrganizationKey,ParentOrganizationKey,PercentageOfOwnership,OrganizationName,CurrencyKey,CurrencyAlternateKey,CurrencyName
0,1,NaN,1,AdventureWorks Cycle,100,USD,US Dollar
1,2,1.0,1,North America Operations,100,USD,US Dollar
2,3,14.0,1,Northeast Division,100,USD,US Dollar
3,4,14.0,1,Northwest Division,100,USD,US Dollar
4,5,14.0,1,Central Division,100,USD,US Dollar


,OrganizationKey,ParentOrganizationKey,PercentageOfOwnership,OrganizationName,CurrencyName
0,1,NaN,1,AdventureWorks Cycle,US Dollar
1,2,1.0,1,North America Operations,US Dollar
2,3,14.0,1,Northeast Division,US Dollar
3,4,14.0,1,Northwest Division,US Dollar
4,5,14.0,1,Central Division,US Dollar
5,6,14.0,1,Southeast Division,US Dollar
6,7,14.0,1,Southwest Division,US Dollar
7,8,2.0,.75,Canadian Division,Canadian Dollar
8,9,1.0,1,European Operations,EURO
9,10,1.0,.75,Pacific Operations,Australian Dollar


In [42]:
#Trazer o nome das organizações 'pais' invés do código
DimOrganization_trat = DimOrganization_trat.merge(DimOrganization, how='left', left_on='ParentOrganizationKey',right_on='OrganizationKey')
DimOrganization_trat

,OrganizationKey_x,ParentOrganizationKey_x,PercentageOfOwnership_x,OrganizationName_x,CurrencyKey_x,CurrencyAlternateKey,CurrencyName,OrganizationKey_y,ParentOrganizationKey_y,PercentageOfOwnership_y,OrganizationName_y,CurrencyKey_y
0,1,NaN,1,AdventureWorks Cycle,100,USD,US Dollar,NaN,NaN,NaN,NaN,NaN
1,2,1.0,1,North America Operations,100,USD,US Dollar,1.0,NaN,1,AdventureWorks Cycle,100.0
2,3,14.0,1,Northeast Division,100,USD,US Dollar,14.0,2.0,1,USA Operations,100.0
3,4,14.0,1,Northwest Division,100,USD,US Dollar,14.0,2.0,1,USA Operations,100.0
4,5,14.0,1,Central Division,100,USD,US Dollar,14.0,2.0,1,USA Operations,100.0
5,6,14.0,1,Southeast Division,100,USD,US Dollar,14.0,2.0,1,USA Operations,100.0
6,7,14.0,1,Southwest Division,100,USD,US Dollar,14.0,2.0,1,USA Operations,100.0
7,8,2.0,.75,Canadian Division,19,CAD,Canadian Dollar,2.0,1.0,1,North America Operations,100.0
8,9,1.0,1,European Operations,36,EUR,EURO,1.0,NaN,1,AdventureWorks Cycle,100.0
9,10,1.0,.75,Pacific Operations,6,AUD,Australian Dollar,1.0,NaN,1,AdventureWorks Cycle,100.0


In [43]:
#Seleção das colunas
DimOrganization_trat = DimOrganization_trat[["OrganizationKey_x","OrganizationName_x","OrganizationName_y","PercentageOfOwnership_x","CurrencyName"]]
DimOrganization_trat.head()

,OrganizationKey_x,OrganizationName_x,OrganizationName_y,PercentageOfOwnership_x,CurrencyName
0,1,AdventureWorks Cycle,NaN,1,US Dollar
1,2,North America Operations,AdventureWorks Cycle,1,US Dollar
2,3,Northeast Division,USA Operations,1,US Dollar
3,4,Northwest Division,USA Operations,1,US Dollar
4,5,Central Division,USA Operations,1,US Dollar


In [38]:
DimOrganization_trat

,OrganizationKey_x,ParentOrganizationKey_x,PercentageOfOwnership_x,OrganizationName_x,CurrencyKey_x,CurrencyAlternateKey,CurrencyName,OrganizationKey_y,ParentOrganizationKey_y,PercentageOfOwnership_y,OrganizationName_y,CurrencyKey_y
0,1,NaN,1,AdventureWorks Cycle,100,USD,US Dollar,NaN,NaN,NaN,NaN,NaN
1,2,1.0,1,North America Operations,100,USD,US Dollar,1.0,NaN,1,AdventureWorks Cycle,100.0
2,3,14.0,1,Northeast Division,100,USD,US Dollar,14.0,2.0,1,USA Operations,100.0
3,4,14.0,1,Northwest Division,100,USD,US Dollar,14.0,2.0,1,USA Operations,100.0
4,5,14.0,1,Central Division,100,USD,US Dollar,14.0,2.0,1,USA Operations,100.0
5,6,14.0,1,Southeast Division,100,USD,US Dollar,14.0,2.0,1,USA Operations,100.0
6,7,14.0,1,Southwest Division,100,USD,US Dollar,14.0,2.0,1,USA Operations,100.0
7,8,2.0,.75,Canadian Division,19,CAD,Canadian Dollar,2.0,1.0,1,North America Operations,100.0
8,9,1.0,1,European Operations,36,EUR,EURO,1.0,NaN,1,AdventureWorks Cycle,100.0
9,10,1.0,.75,Pacific Operations,6,AUD,Australian Dollar,1.0,NaN,1,AdventureWorks Cycle,100.0


In [45]:
#Renomeação das colunas
DimOrganization_trat = DimOrganization_trat.rename(columns={"OrganizationKey_x":"OrganizationKey","OrganizationName_x":"OrganizationName","OrganizationName_y":"ParentOrganizationName","PercentageOfOwnership_x":"PercentageOfOwnership"})
DimOrganization_trat.head()

,OrganizationKey,OrganizationName,ParentOrganizationName,PercentageOfOwnership,CurrencyName
0,1,AdventureWorks Cycle,NaN,1,US Dollar
1,2,North America Operations,AdventureWorks Cycle,1,US Dollar
2,3,Northeast Division,USA Operations,1,US Dollar
3,4,Northwest Division,USA Operations,1,US Dollar
4,5,Central Division,USA Operations,1,US Dollar


In [46]:
DimDepartmentGroup.head()

,DepartmentGroupKey,ParentDepartmentGroupKey,DepartmentGroupName
0,1,NaN,Corporate
1,2,1.0,Executive General and Administration
2,3,1.0,Inventory Management
3,4,1.0,Manufacturing
4,5,1.0,Quality Assurance


### Verifico que a DimDepartmentGroup também possui o código do "pai" invés do nome

In [47]:
#Trazer o nome dos grupos de departamentos 'pais' invés do código
DimDepartmentGroup_trat = DimDepartmentGroup.merge(DimDepartmentGroup, how='left', left_on='ParentDepartmentGroupKey',right_on='DepartmentGroupKey')
DimDepartmentGroup_trat

,DepartmentGroupKey_x,ParentDepartmentGroupKey_x,DepartmentGroupName_x,DepartmentGroupKey_y,ParentDepartmentGroupKey_y,DepartmentGroupName_y
0,1,NaN,Corporate,NaN,NaN,NaN
1,2,1.0,Executive General and Administration,1.0,NaN,Corporate
2,3,1.0,Inventory Management,1.0,NaN,Corporate
3,4,1.0,Manufacturing,1.0,NaN,Corporate
4,5,1.0,Quality Assurance,1.0,NaN,Corporate
5,6,1.0,Research and Development,1.0,NaN,Corporate
6,7,1.0,Sales and Marketing,1.0,NaN,Corporate


In [48]:
#Seleção das colunas
DimDepartmentGroup_trat = DimDepartmentGroup_trat[["DepartmentGroupKey_x","DepartmentGroupName_x","DepartmentGroupName_y"]]
DimDepartmentGroup_trat.head()

,DepartmentGroupKey_x,DepartmentGroupName_x,DepartmentGroupName_y
0,1,Corporate,NaN
1,2,Executive General and Administration,Corporate
2,3,Inventory Management,Corporate
3,4,Manufacturing,Corporate
4,5,Quality Assurance,Corporate


In [49]:
#Renomeação das colunas
DimDepartmentGroup_trat = DimDepartmentGroup_trat.rename(columns={"DepartmentGroupKey_x":"DepartmentGroupKey","DepartmentGroupName_x":"DepartmentGroupName","DepartmentGroupName_y":"ParentDepartmentGroupName"})
DimDepartmentGroup_trat.head()

,DepartmentGroupKey,DepartmentGroupName,ParentDepartmentGroupName
0,1,Corporate,NaN
1,2,Executive General and Administration,Corporate
2,3,Inventory Management,Corporate
3,4,Manufacturing,Corporate
4,5,Quality Assurance,Corporate


In [51]:
DimAccount.head()

,AccountKey,ParentAccountKey,AccountCodeAlternateKey,ParentAccountCodeAlternateKey,AccountDescription,AccountType,Operator,CustomMembers,ValueType,CustomMemberOptions
0,1,NaN,1,NaN,Balance Sheet,None,~,None,Currency,None
1,2,1.0,10,1.0,Assets,Assets,+,None,Currency,None
2,3,2.0,110,10.0,Current Assets,Assets,+,None,Currency,None
3,4,3.0,1110,110.0,Cash,Assets,+,None,Currency,None
4,5,3.0,1120,110.0,Receivables,Assets,+,None,Currency,None


In [112]:
#Removeção de colunas desnecessárias pois não trazem informação
DimAccount_trat = DimAccount[['AccountKey','ParentAccountKey','AccountDescription','AccountType','Operator','ValueType']]

### Verifico que a DimAccount também possui o código do "pai" invés do nome

### Investigarei o caracter ~ na coluna Operator, o padrão seria indicar uma operação positiva ou negativa

In [55]:
DimAccount.Operator.unique()

array(['~', '+', '-'], dtype=object)

In [64]:
DimAccount[DimAccount['Operator'] == '~']

,AccountKey,ParentAccountKey,AccountCodeAlternateKey,ParentAccountCodeAlternateKey,AccountDescription,AccountType,Operator,CustomMembers,ValueType,CustomMemberOptions
0,1,NaN,1,NaN,Balance Sheet,None,~,None,Currency,None
92,95,NaN,9500,NaN,Statistical Accounts,Statistical,~,None,Units,None
93,96,95.0,9510,9500.0,Headcount,Balances,~,None,Units,None
94,97,95.0,9520,9500.0,Units,Flow,~,None,Units,None
95,98,95.0,9530,9500.0,Average Unit Price,Balances,~,[Account].[Accounts].[Account Level 04].&[50]/...,Currency,None
96,99,95.0,9540,9500.0,Square Footage,Balances,~,None,Units,None


In [65]:
FactFinance[FactFinance['AccountKey'] == 1]

,FinanceKey,DateKey,OrganizationKey,DepartmentGroupKey,ScenarioKey,AccountKey,Amount,Date


In [67]:
FactFinance[FactFinance['AccountKey'] == 95]

,FinanceKey,DateKey,OrganizationKey,DepartmentGroupKey,ScenarioKey,AccountKey,Amount,Date


In [74]:
FactFinance[FactFinance['AccountKey'] == 93].head()

,FinanceKey,DateKey,OrganizationKey,DepartmentGroupKey,ScenarioKey,AccountKey,Amount,Date
13474,13475,20111228,3,7,1,93,16875.0,2011-12-28
13477,13478,20111228,4,7,1,93,-5370.0,2011-12-28
13480,13481,20111228,5,7,1,93,15622.0,2011-12-28
13483,13484,20111228,6,7,1,93,33105.0,2011-12-28
13486,13487,20111228,7,7,1,93,24633.0,2011-12-28


### Entre as Accounts que tem operator ~, 2 não apresentaram registro na tabela fato, suponho que seja por serem 'pais'. As outras Accounts tem o ValueType como Units.

In [75]:
DimAccount[DimAccount['ValueType'] == 'Units']

,AccountKey,ParentAccountKey,AccountCodeAlternateKey,ParentAccountCodeAlternateKey,AccountDescription,AccountType,Operator,CustomMembers,ValueType,CustomMemberOptions
92,95,NaN,9500,NaN,Statistical Accounts,Statistical,~,None,Units,None
93,96,95.0,9510,9500.0,Headcount,Balances,~,None,Units,None
94,97,95.0,9520,9500.0,Units,Flow,~,None,Units,None
96,99,95.0,9540,9500.0,Square Footage,Balances,~,None,Units,None


### Segundo a descrição destes registros indicam que contém informações diferentes, como metro quadrado. Irei desconsidera-los pois não se referem a Financeiro.

In [108]:
#Identificar quais códigos de conta tem operador ~
regRemov = DimAccount.AccountKey[DimAccount['Operator'] == '~'].tolist()

[1, 95, 96, 97, 98, 99]

In [111]:
#Removendo na fato os registros referentes a estas accounts
FactFinance_trat = FactFinance[~FactFinance['AccountKey'].isin(regRemov)]

In [128]:
DimAccount_trat[DimAccount_trat['ParentAccountKey'].isna()]

,AccountKey,ParentAccountKey,AccountDescription,AccountType,Operator,ValueType
0,1,NaN,Balance Sheet,None,~,Currency
45,47,NaN,Net Income,Revenue,+,Currency
92,95,NaN,Statistical Accounts,Statistical,~,Units


### Verifico que a DimAccount também possui o código do "pai" invés do nome

In [50]:
DimScenario.head()

,ScenarioKey,ScenarioName
0,1,Actual
1,2,Budget
2,3,Forecast


### Ter a informações contidas na tabela DimScenario como uma tabela separada é desnecessário e irei levar para a FactFinance